## SAM 3: Segmentação de Imagem e Propagação em Vídeo com Exportação COCO

Este notebook demonstra como:

1. **Segmentar uma imagem** usando prompts de texto ou visuais
2. **Propagar as anotações** para um vídeo completo
3. **Visualizar todos os frames** com as anotações
4. **Salvar as anotações** em formato COCO JSON

In [ ]:
using_colab = False

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib scikit-learn
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam3.git'

## Setup e Imports

In [ ]:
import os
import json
import glob
from datetime import datetime

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image

import sam3
from sam3 import build_sam3_image_model
from sam3.model_builder import build_sam3_video_predictor
from sam3.model.box_ops import box_xywh_to_cxcywh
from sam3.model.sam3_image_processor import Sam3Processor
from sam3.visualization_utils import (
    draw_box_on_image,
    load_frame,
    normalize_bbox,
    plot_results,
    prepare_masks_for_visualization,
    visualize_formatted_frame_output,
)

sam3_root = os.path.join(os.path.dirname(sam3.__file__), "..")

# Configurar matplotlib
plt.rcParams["axes.titlesize"] = 12
plt.rcParams["figure.titlesize"] = 12

In [ ]:
# Habilitar tf32 para GPUs Ampere
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

# Usar bfloat16 para todo o notebook
torch.autocast("cuda", dtype=torch.bfloat16).__enter__()

## Funções Auxiliares

In [ ]:
def abs_to_rel_coords(coords, IMG_WIDTH, IMG_HEIGHT, coord_type="point"):
    """Converte coordenadas absolutas para relativas (0-1)
    
    Args:
        coords: Lista de coordenadas
        coord_type: 'point' para [x, y] ou 'box' para [x, y, w, h]
    """
    if coord_type == "point":
        return [[x / IMG_WIDTH, y / IMG_HEIGHT] for x, y in coords]
    elif coord_type == "box":
        return [
            [x / IMG_WIDTH, y / IMG_HEIGHT, w / IMG_WIDTH, h / IMG_HEIGHT]
            for x, y, w, h in coords
        ]
    else:
        raise ValueError(f"Unknown coord_type: {coord_type}")


def propagate_in_video(predictor, session_id):
    """Propaga as anotações do frame inicial para todo o vídeo"""
    outputs_per_frame = {}
    for response in predictor.handle_stream_request(
        request=dict(
            type="propagate_in_video",
            session_id=session_id,
        )
    ):
        outputs_per_frame[response["frame_index"]] = response["outputs"]
    return outputs_per_frame


def mask_to_rle(binary_mask):
    """Converte máscara binária para formato RLE (Run Length Encoding)"""
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    
    # Flatten mask
    flat_mask = binary_mask.ravel(order='F')
    
    # Encode RLE
    last_val = 0
    count = 0
    for val in flat_mask:
        if val != last_val:
            counts.append(count)
            count = 1
            last_val = val
        else:
            count += 1
    counts.append(count)
    
    return rle


def mask_to_bbox(binary_mask):
    """Extrai bounding box de uma máscara binária no formato COCO [x, y, width, height]"""
    rows = np.any(binary_mask, axis=1)
    cols = np.any(binary_mask, axis=0)
    
    if not rows.any() or not cols.any():
        return [0, 0, 0, 0]
    
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    
    return [int(cmin), int(rmin), int(cmax - cmin + 1), int(rmax - rmin + 1)]


def calculate_mask_area(binary_mask):
    """Calcula a área de uma máscara binária"""
    return int(np.sum(binary_mask))

## Parte 1: Segmentação de Imagem com Prompt

In [ ]:
# Construir modelo de imagem
bpe_path = f"{sam3_root}/assets/bpe_simple_vocab_16e6.txt.gz"
image_model = build_sam3_image_model(bpe_path=bpe_path)
print("Modelo de imagem carregado com sucesso!")

In [ ]:
# Carregar imagem
image_path = f"{sam3_root}/assets/images/test_image.jpg"
image = Image.open(image_path)
width, height = image.size
print(f"Imagem carregada: {width}x{height}")

# Configurar processador
processor = Sam3Processor(image_model, confidence_threshold=0.5)
inference_state = processor.set_image(image)

In [ ]:
# Segmentar com prompt de texto
# Você pode alterar o prompt aqui
prompt_text = "monitor"

processor.reset_all_prompts(inference_state)
inference_state = processor.set_text_prompt(state=inference_state, prompt=prompt_text)

# Visualizar resultados
img0 = Image.open(image_path)
plot_results(img0, inference_state)
print(f"Segmentação completa com prompt: '{prompt_text}'")

### Alternativa: Usar Prompt Visual (Box)

Descomente o código abaixo se preferir usar um bounding box ao invés de texto:

In [ ]:
# # Exemplo com bounding box (formato x, y, w, h)
# box_input_xywh = torch.tensor([480.0, 290.0, 110.0, 360.0]).view(-1, 4)
# box_input_cxcywh = box_xywh_to_cxcywh(box_input_xywh)
# norm_box_cxcywh = normalize_bbox(box_input_cxcywh, width, height).flatten().tolist()

# processor.reset_all_prompts(inference_state)
# inference_state = processor.add_geometric_prompt(
#     state=inference_state, box=norm_box_cxcywh, label=True
# )

# # Visualizar com box
# image_with_box = draw_box_on_image(img0, box_input_xywh.flatten().tolist())
# plt.figure(figsize=(10, 8))
# plt.imshow(image_with_box)
# plt.axis("off")
# plt.title("Imagem com Bounding Box")
# plt.show()

# plot_results(img0, inference_state)

---

## ⚡ ATALHO: Usar Primeiro Frame do Vídeo

**Se você quer segmentar o PRIMEIRO FRAME DO VÍDEO e propagar**, siga estes passos:

1. **Execute primeiro** as células da "Parte 2: Propagação em Vídeo" para carregar o vídeo
2. **Depois volte aqui** e use o código abaixo:

```python
# Use este código DEPOIS de carregar o vídeo na Parte 2
if isinstance(video_frames_for_vis[0], str):
    first_frame = Image.open(video_frames_for_vis[0])
else:
    first_frame = Image.fromarray(video_frames_for_vis[0])

# Reconfigure o processador com o primeiro frame
inference_state = processor.set_image(first_frame)
width, height = first_frame.size

# Agora segmente este frame (use o mesmo código de prompt/box acima)
```

3. **Continue com a Parte 2** normalmente, usando o **mesmo** `prompt_text`

---

## Parte 2: Propagação em Vídeo

In [ ]:
# Carregar vídeo
video_path = f"{sam3_root}/assets/videos/0001"

# Carregar frames para visualização
if isinstance(video_path, str) and video_path.endswith(".mp4"):
    cap = cv2.VideoCapture(video_path)
    video_frames_for_vis = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        video_frames_for_vis.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    cap.release()
else:
    video_frames_for_vis = glob.glob(os.path.join(video_path, "*.jpg"))
    try:
        video_frames_for_vis.sort(
            key=lambda p: int(os.path.splitext(os.path.basename(p))[0])
        )
    except ValueError:
        print(f"Frames não estão no formato '<frame_index>.jpg', usando ordem lexicográfica")
        video_frames_for_vis.sort()

print(f"Vídeo carregado com {len(video_frames_for_vis)} frames")

In [ ]:
# Iniciar sessão de inferência
response = video_predictor.handle_request(
    request=dict(
        type="start_session",
        resource_path=video_path,
    )
)
session_id = response["session_id"]
print(f"Sessão iniciada: {session_id}")

In [ ]:
# Extrair o primeiro frame do vídeo para segmentação inicial
if isinstance(video_frames_for_vis[0], str):
    first_frame = Image.open(video_frames_for_vis[0])
    first_frame_array = np.array(first_frame)
else:
    first_frame_array = video_frames_for_vis[0]
    first_frame = Image.fromarray(first_frame_array)

width, height = first_frame.size
print(f"Primeiro frame extraído: {width}x{height}")

# Visualizar o primeiro frame
plt.figure(figsize=(10, 8))
plt.imshow(first_frame_array)
plt.axis("off")
plt.title("Frame 0 - Frame Inicial do Vídeo")
plt.show()

## Parte 2: Segmentar o Primeiro Frame com Prompt

⚠️ **IMPORTANTE**: Aqui segmentamos o **primeiro frame do vídeo (frame 0)**.
Essa segmentação será depois propagada para todos os outros frames.

In [ ]:
# Construir modelo de imagem
bpe_path = f"{sam3_root}/assets/bpe_simple_vocab_16e6.txt.gz"
image_model = build_sam3_image_model(bpe_path=bpe_path)
print("Modelo de imagem carregado com sucesso!")

## Parte 4: Visualização de Todos os Frames

In [ ]:
# Segmentar o primeiro frame com prompt de texto
# 🎯 Você pode alterar o prompt aqui
prompt_text = "person"

processor.reset_all_prompts(inference_state)
inference_state = processor.set_text_prompt(state=inference_state, prompt=prompt_text)

# Visualizar resultados da segmentação
plot_results(first_frame, inference_state)
print(f"✅ Segmentação do frame 0 completa com prompt: '{prompt_text}'")

## Parte 5: Exportar Anotações para Formato COCO JSON

In [ ]:
# # Exemplo com bounding box (formato x, y, w, h)
# # Ajuste as coordenadas para o seu primeiro frame
# box_input_xywh = torch.tensor([480.0, 290.0, 110.0, 360.0]).view(-1, 4)
# box_input_cxcywh = box_xywh_to_cxcywh(box_input_xywh)
# norm_box_cxcywh = normalize_bbox(box_input_cxcywh, width, height).flatten().tolist()

# processor.reset_all_prompts(inference_state)
# inference_state = processor.add_geometric_prompt(
#     state=inference_state, box=norm_box_cxcywh, label=True
# )

# # Visualizar com box
# image_with_box = draw_box_on_image(first_frame, box_input_xywh.flatten().tolist())
# plt.figure(figsize=(10, 8))
# plt.imshow(image_with_box)
# plt.axis("off")
# plt.title("Frame 0 com Bounding Box")
# plt.show()

# plot_results(first_frame, inference_state)

## Parte 3: Propagar Segmentação para Todo o Vídeo

🔄 Agora vamos propagar a segmentação do frame 0 para todos os frames do vídeo.

In [ ]:
# Configurar GPUs
gpus_to_use = range(torch.cuda.device_count())
# Para usar apenas uma GPU:
# gpus_to_use = [torch.cuda.current_device()]

# Construir preditor de vídeo
video_predictor = build_sam3_video_predictor(gpus_to_use=gpus_to_use)
print(f"Preditor de vídeo construído com {len(gpus_to_use)} GPU(s)")

In [ ]:
# Adicionar prompt de texto no frame 0
# Use o mesmo prompt da segmentação de imagem ou altere conforme necessário
video_prompt_text = "person"
frame_idx = 0

response = video_predictor.handle_request(
    request=dict(
        type="add_prompt",
        session_id=session_id,
        frame_index=frame_idx,
        text=video_prompt_text,
    )
)
out = response["outputs"]

# Visualizar resultado no frame inicial
plt.close("all")
visualize_formatted_frame_output(
    frame_idx,
    video_frames_for_vis,
    outputs_list=[prepare_masks_for_visualization({frame_idx: out})],
    titles=[f"Frame {frame_idx} com prompt: '{video_prompt_text}'"],
    figsize=(8, 6),
)
print(f"Prompt '{video_prompt_text}' adicionado no frame {frame_idx}")

In [ ]:
# Propagar anotações para todo o vídeo
print("Propagando anotações por todo o vídeo...")
outputs_per_frame = propagate_in_video(video_predictor, session_id)
print(f"Propagação completa! {len(outputs_per_frame)} frames processados")

## Parte 3: Visualização de Todos os Frames

In [ ]:
# Preparar máscaras para visualização
formatted_outputs = prepare_masks_for_visualization(outputs_per_frame)

# Visualizar TODOS os frames (pode gerar muitas imagens!)
# Para vídeos longos, considere usar um stride maior
vis_frame_stride = 1  # Altere para 5, 10, etc. para pular frames

print(f"Visualizando frames com stride={vis_frame_stride}")
plt.close("all")

for frame_idx in range(0, len(formatted_outputs), vis_frame_stride):
    visualize_formatted_frame_output(
        frame_idx,
        video_frames_for_vis,
        outputs_list=[formatted_outputs],
        titles=[f"SAM 3 - Frame {frame_idx}"],
        figsize=(8, 6),
    )

print(f"Visualização completa de {len(range(0, len(formatted_outputs), vis_frame_stride))} frames")

## Parte 4: Exportar Anotações para Formato COCO JSON

In [ ]:
def export_to_coco_json(outputs_per_frame, video_frames, output_path, video_name="video"):
    """
    Exporta as anotações do SAM 3 para formato COCO JSON
    
    Args:
        outputs_per_frame: Dicionário com outputs por frame
        video_frames: Lista de paths ou arrays dos frames
        output_path: Caminho para salvar o arquivo JSON
        video_name: Nome do vídeo
    """
    # Estrutura base COCO
    coco_data = {
        "info": {
            "description": "SAM 3 Video Annotations",
            "version": "1.0",
            "year": datetime.now().year,
            "date_created": datetime.now().isoformat(),
        },
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": [],
    }
    
    # Obter dimensões do frame
    if isinstance(video_frames[0], str):
        first_frame = np.array(Image.open(video_frames[0]))
    else:
        first_frame = video_frames[0]
    
    height, width = first_frame.shape[:2]
    
    # Coletar todas as categorias (object IDs) únicas
    all_object_ids = set()
    for frame_data in outputs_per_frame.values():
        for obj_id in frame_data.keys():
            all_object_ids.add(obj_id)
    
    # Criar categorias
    for obj_id in sorted(all_object_ids):
        coco_data["categories"].append({
            "id": int(obj_id),
            "name": f"object_{obj_id}",
            "supercategory": "object",
        })
    
    annotation_id = 1
    
    # Processar cada frame
    for frame_idx in sorted(outputs_per_frame.keys()):
        # Adicionar informação da imagem
        image_id = frame_idx + 1
        if isinstance(video_frames[frame_idx], str):
            file_name = os.path.basename(video_frames[frame_idx])
        else:
            file_name = f"{video_name}_frame_{frame_idx:05d}.jpg"
        
        coco_data["images"].append({
            "id": image_id,
            "file_name": file_name,
            "height": height,
            "width": width,
            "frame_index": frame_idx,
        })
        
        # Processar cada objeto no frame
        frame_data = outputs_per_frame[frame_idx]
        for obj_id, obj_data in frame_data.items():
            # Extrair máscara
            if isinstance(obj_data, dict) and "mask" in obj_data:
                mask = obj_data["mask"]
            else:
                mask = obj_data
            
            # Converter para numpy array se necessário
            if torch.is_tensor(mask):
                mask = mask.cpu().numpy()
            
            # Garantir que a máscara é binária
            binary_mask = (mask > 0).astype(np.uint8)
            
            # Calcular bbox e área
            bbox = mask_to_bbox(binary_mask)
            area = calculate_mask_area(binary_mask)
            
            # Pular se a máscara estiver vazia
            if area == 0:
                continue
            
            # Converter máscara para RLE
            rle = mask_to_rle(binary_mask)
            
            # Adicionar anotação
            coco_data["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": int(obj_id),
                "segmentation": rle,
                "area": area,
                "bbox": bbox,
                "iscrowd": 0,
            })
            
            annotation_id += 1
    
    # Salvar JSON
    with open(output_path, 'w') as f:
        json.dump(coco_data, f, indent=2)
    
    print(f"\nAnotações COCO salvas em: {output_path}")
    print(f"  - Total de imagens: {len(coco_data['images'])}")
    print(f"  - Total de anotações: {len(coco_data['annotations'])}")
    print(f"  - Total de categorias: {len(coco_data['categories'])}")
    
    return coco_data

In [ ]:
# Exportar para COCO JSON
output_json_path = f"{sam3_root}/outputs/annotations_coco.json"

# Criar diretório de saída se não existir
os.makedirs(os.path.dirname(output_json_path), exist_ok=True)

# Exportar
coco_annotations = export_to_coco_json(
    outputs_per_frame=outputs_per_frame,
    video_frames=video_frames_for_vis,
    output_path=output_json_path,
    video_name="sam3_video"
)

print("\nExportação completa!")

## Verificar Anotações Exportadas

In [ ]:
# Carregar e exibir estatísticas do arquivo COCO
with open(output_json_path, 'r') as f:
    coco_data = json.load(f)

print("\n=== Estatísticas das Anotações COCO ===")
print(f"Número de frames anotados: {len(coco_data['images'])}")
print(f"Número total de anotações: {len(coco_data['annotations'])}")
print(f"Número de categorias/objetos: {len(coco_data['categories'])}")
print("\nCategorias detectadas:")
for cat in coco_data['categories']:
    cat_annotations = [ann for ann in coco_data['annotations'] if ann['category_id'] == cat['id']]
    print(f"  - {cat['name']} (ID: {cat['id']}): {len(cat_annotations)} anotações")

## Limpeza e Encerramento

In [ ]:
# Fechar sessão de inferência
_ = video_predictor.handle_request(
    request=dict(
        type="close_session",
        session_id=session_id,
    )
)
print("Sessão encerrada")

In [ ]:
# Desligar preditor
video_predictor.shutdown()
print("Preditor desligado")

## Resumo

Este notebook demonstrou:

✅ **Segmentação de imagem** com prompt de texto ou visual  
✅ **Propagação de anotações** em vídeo completo  
✅ **Visualização de todos os frames** com as máscaras  
✅ **Exportação para COCO JSON** com RLE, bboxes e áreas  

O arquivo COCO gerado pode ser usado para:
- Treinamento de modelos de detecção/segmentação
- Avaliação de performance
- Análise de objetos ao longo do tempo
- Integração com outras ferramentas de visão computacional